# Nonparallel code

We'll start with nonparallelized code. Unless told otherwise, a program generally runs on one computing *core*. A core is a unit of the computer that executes instructions. Most modern computers have multiple cores, which enables the computer to execute sets of instructions simultaneously, or parallel processing. 

Due to limitations in how chips are produced and how much heat they can tolerate, computing cores have not gotten signifcantly faster for many years. The way to speed up computing has, instead, been to provide multiple cores allowing for parallel processing. If you can't make the cores faster, give the user more of them. 

We'll start with the typical case of a program running on one core and then we'll move up to parallel processing on different systems. 

Our example will be computing *Euler's constant* $e$. Julia (and most languages) has the `exp(n)` function that returns $e^n$. 


In [7]:
# This cell should add the packages you need for 
# this notebook (and other notebooks in this directory)
# You should get a message saying 
# "Activating project at `.../TryMPI/notebooks` where the ... is the 
# absolute path to the directory of this repository and these notebooks.
using Pkg
Pkg.activate(".")
Pkg.instantiate()

# This cell will appear in other notebooks as well

  Activating project at `~/Development/HighVelocityJuliaAnalysis/TryMPI/notebooks`


In [2]:
exp(1)

2.718281828459045

One way to compute $e$ by hand is with a series expansion...

$$e = \sum^\infty_{i=0} \frac{1}{i!}$$

where $i!$ is the *factorial* $i! = 1 \times 2 \times ... \times i-2 \times i-1 \times i$

Julia has the `factorial(i)` function that returns `i!`.

In [4]:
factorial(5)

120

In [6]:
factorial(5) == 1*2*3*4*5

true

Let's write a simple function that will calculate a *term* of the expansion.

In [7]:
calculateTerm(i) = 1.0/factorial(i)

calculateTerm (generic function with 1 method)

Let's see how close we get to $e$ with 10 terms

In [12]:
tenTerms = [ calculateTerm(i) for i in 0:10 ] |> sum

2.7182818011463845

Remember the *list comprehension* style...

```julia
[ calculateTerm(i) for i in 0:10 ]
```
is equivalent to
```julia
result = []
for i in 0:10
  push!(result, calculateTerm(i))
end
```

And remember the "pipe" operator |> ... 
```julia
[1,2,3] |> sum
```
is equivalent to
```julia
sum([1,2,3])
```

How close did we get? ...

In [13]:
exp(1) - tenTerms

2.7312660577649694e-8

With just 10 terms, we get really close. Let's put this calculation in it's own function. 

In [35]:
calculate_e(nTerms) = [ calculateTerm(i) for i in 0:nTerms ] |> sum

calculate_e (generic function with 1 method)

In [36]:
twentyTerms = calculate_e(20)

2.7182818284590455

In [37]:
exp(1) - twentyTerms

-4.440892098500626e-16

Extremely close!

Let's try benchmarking this code.

In [29]:
using Pkg
Pkg.activate("notebooks")

using BenchmarkTools

  Activating project at `~/Development/HighVelocityJuliaAnalysis/TryMPI/notebooks`


In [42]:
@benchmark calculate_e(20)

BenchmarkTools.Trial: 10000 samples with 963 evaluations.
 Range (min … max):  65.535 ns … 663.666 ns  ┊ GC (min … max): 0.00% … 64.94%
 Time  (median):     81.559 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   86.384 ns ±  31.321 ns  ┊ GC (mean ± σ):  2.07% ±  5.27%

  ▄▂ ▁▁▂▁▄▅█▇▆▄▂▂▂▁▂▂▁     ▃                                   ▂
  ██████████████████████▇▇▇██▇▇▇▆▆▆▆▆▅▆▆▆▅▅▆▄▄▅▄▄▅▄▆▆▅▅▄▂▄▅▄▄▅ █
  65.5 ns       Histogram: log(frequency) by time       163 ns <

 Memory estimate: 224 bytes, allocs estimate: 1.

This code runs really fast! But we can make it run faster with parallelization. 